<a href="https://colab.research.google.com/github/patilaalap/Signature_forgery/blob/main/Sign_detc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

##Fetching Kaggle Dataset

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Colab Notebooks"

In [4]:
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
!kaggle datasets download -d robinreni/signature-verification-dataset

 98% 589M/601M [00:17<00:00, 32.7MB/s]
100% 601M/601M [00:17<00:00, 35.9MB/s]


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

Archive:  signature-verification-dataset.zip
  inflating: sign_data/sign_data/test/049/01_049.png  
  inflating: sign_data/sign_data/test/049/02_049.png  
  inflating: sign_data/sign_data/test/049/03_049.png  
  inflating: sign_data/sign_data/test/049/04_049.png  
  inflating: sign_data/sign_data/test/049/05_049.png  
  inflating: sign_data/sign_data/test/049/06_049.png  
  inflating: sign_data/sign_data/test/049/07_049.png  
  inflating: sign_data/sign_data/test/049/08_049.png  
  inflating: sign_data/sign_data/test/049/09_049.png  
  inflating: sign_data/sign_data/test/049/10_049.png  
  inflating: sign_data/sign_data/test/049/11_049.png  
  inflating: sign_data/sign_data/test/049/12_049.png  
  inflating: sign_data/sign_data/test/049_forg/01_0114049.PNG  
  inflating: sign_data/sign_data/test/049_forg/01_0206049.PNG  
  inflating: sign_data/sign_data/test/049_forg/01_0210049.PNG  
  inflating: sign_data/sign_data/test/049_forg/02_0114049.PNG  
  inflating: sign_data/sign_data/test/0

##Classifying Data

In [ ]:
!mkdir Train
!mkdir Test

!mkdir Train/Fake
!mkdir Train/Real

!mkdir Test/Fake
!mkdir Test/Real

In [ ]:
PATH = "sign_data/train/"

for i in os.listdir(PATH):
    
    contol = i.split("_")
    
    try:
    
        if contol[1]=="forg":
            
            os.system("cp -r {} Train/Fake".format(PATH+i))
            
    except:
        
        os.system("cp -r {} Train/Real".format(PATH+i))

In [ ]:
PATH = "sign_data/test/"

for i in os.listdir(PATH):
    
    contol = i.split("_")
    
    try:
    
        if contol[1]=="forg":
            
            os.system("cp -r {} Test/Fake".format(PATH+i))
            
    except:
        
        os.system("cp -r {} Test/Real".format(PATH+i))

In [5]:
train_dir = os.path.join("Train") 

test_dir = os.path.join("Test")

In [6]:
train_datagen = ImageDataGenerator(

      # compressing picture pixels between 0.1
      rescale=1./255,

      # Angle of random rotation of images in degrees (0-180)
      rotation_range=40,

      # horizontal and vertical scrolling rates of images
      width_shift_range=0.2,
      height_shift_range=0.2,

      # buckling process
      shear_range=0.2,

      # zoom operation
      zoom_range=0.2,

      # rotate image vertically
      horizontal_flip=True,

      # excess after processing
      # determines how to fill image points
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(

        # target directory
        train_dir,

        # all images will be resized as (150x150)
        target_size=(200, 200),

        # stack size
        batch_size=64,

        # Since we are using binary_crossentropy
        # binary tags are required.
        class_mode='binary')

Found 1649 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(

        test_dir,

        target_size=(200, 200),

        batch_size=64,

        class_mode='binary')

Found 500 images belonging to 2 classes.


##Model

In [9]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    
    self.cnn1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(3,200,200))
    self.cnn2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')
    self.cnn3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')
    
    self.flatten = tf.keras.layers.Flatten()
    
    self.dense1 = tf.keras.layers.Dense(512, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    
    x = self.cnn1(inputs)
    x = self.cnn2(x)
    x = self.cnn3(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

model = MyModel()

In [10]:
input_shape = (None, 200, 200, 3)
model.build(input_shape)
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  448       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4640      
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  4624      
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  308314624 
_________________________________________________________________
dense_1 (Dense)              multiple                  513       
Total params: 308,324,849
Trainable params: 308,324,849
Non-trainable params: 0
____________________________________________

In [11]:
model.compile(
    # loss function
    loss="binary_crossentropy",
    
    # optimization:
    # Considering the loss created by the data, which is the input of our network,
    # self-update mechanism
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),

    # metrics to track during training and testing.
    metrics=["acc"])

In [13]:
# We will get the acc, loss, val_acc, val_loss values ​​from the variable named history.
history = model.fit(

    # training data
    train_generator,

    # the number of samples it will run through until the loop finishes (stack to get)
    steps_per_epoch=train_generator.samples//train_generator.batch_size,

    # number of cycles
    epochs=10,

    verbose=2)

Epoch 1/10
25/25 - 268s - loss: 0.6537 - acc: 0.6170
Epoch 2/10
25/25 - 274s - loss: 0.6262 - acc: 0.6637
Epoch 3/10
25/25 - 262s - loss: 0.6337 - acc: 0.6524
Epoch 4/10
25/25 - 270s - loss: 0.6254 - acc: 0.6599
Epoch 5/10
25/25 - 266s - loss: 0.5962 - acc: 0.6795
Epoch 6/10
25/25 - 269s - loss: 0.5926 - acc: 0.6726
Epoch 7/10
25/25 - 261s - loss: 0.5839 - acc: 0.6808
Epoch 8/10
25/25 - 262s - loss: 0.5759 - acc: 0.6959
Epoch 9/10
25/25 - 271s - loss: 0.5617 - acc: 0.7117
Epoch 10/10
25/25 - 268s - loss: 0.5365 - acc: 0.7319


In [14]:
model.evaluate(test_generator)

8/8 [==============================] - 133s 18s/step - loss: 0.5184 - acc: 0.7200


[0.5183759927749634, 0.7200000286102295]

In [15]:
test_input = test_generator[0][0][5]
pred = model.predict(test_input)

if pred>=0.5:
    pred = 1
    
else:
    
    pred = 0
    
print("Predict : ",float(pred))

ValueError: ignored

In [19]:
import os
%pwd

'/content/gdrive/MyDrive/Colab Notebooks'

In [21]:
import pickle
with open('/content/temp', 'w') as fiModel:
    pickle.dump(model, fiModel)

TypeError: ignored

In [22]:
!mkdir -p saved_model
model.save('saved_model/model')

INFO:tensorflow:Assets written to: saved_model/model/assets


In [23]:
new_model = tf.keras.models.load_model('saved_model/model')

# Check its architecture
new_model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  448       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4640      
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  4624      
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  308314624 
_________________________________________________________________
dense_1 (Dense)              multiple                  513       
Total params: 308,324,849
Trainable params: 308,324,849
Non-trainable params: 0
____________________________________________